In [21]:
import numpy as np
import pandas as pd
import matplotlib
from tflearn.datasets import cifar10
from sklearn.neighbors import NearestNeighbors

# Image Classification

## Table of Content
____

* Intro to Image Classification, data-driven approach, pipeline
* Nearest Neighbour Classifer
    * K-Nearest Neighbor
* Validation sets, Cross-validation, hyperparameter tuning
* Pros/ Cons of Nearest Neigbor
* Summary
* Summary : Applying KNN in Practice


__Example:__ For example, in the image below an image classification model takes a single image and assigns probabilities to 4 labels, {cat,dog hat, mug}. As shown in the image, keep in mind that to a computer an image is represented as one large 3-dimensional array of numbers,=. In this example, the cat image is 248 pixels wide, 400 pixels tall, and has three color channels Red,Green,Blue (or RGB for short). Therefore the image consists of 248 x 400 x 3 numbers, or a total of 297,600 numbers. Each number is an integer that ranges from 0 (black) to 255 (white). Our task is to turn this quarter of a million numbers into a single label, such as "cat".

![](http://cs231n.github.io/assets/classify.png)

* The task in Image Classification is to predict a single label (or a distribution over labels as shown here to indicate our confdence) for a given image. Images are 3-dimensional arrays of integers from 0 to 255, of size Width x Height x 3. The three represents the three color channels Red, Green, Blue

## Challenges:

____

AS we present (an exhaustive) list of challenges below, keep in mind the raw represetation of images as a 3-D array of brightness values:

* **Viewpoint Variation**: A single instance of an object can be oriented in mant ways with respect to the camera.

* **Scale Variation**: Visual classes often exhibit variation in their size

* **Deformation**: Many objects of interest are not rigit bodies and can be deformed in extreme ways.

* **Occlusion**: the object of interest can be occuled. Sometimes only a small portion of an object could be visible.

* **Illumination conditions**: The effects of illumination are drastic on the pixel level

* **Background clutter**: The object of interest may blend into their enviroment, making them hard to identify. 

* **Intra-class variation**: The classes of interest can often be relatively voard, such as chair.

## Nearest Neighour Classifier

As our first approach, we will develop what we call a **Nearest Neighbor Classifier**. This classifier has nothing to do with Convolutional Neural Networks, and rarely is used in practice. For this example, we will use the CIFAR-10 dataset.

![](http://cs231n.github.io/assets/nn.jpg)

Suppose now that we are given a CIFAR-10 training set of 50,000 images. and we wish to lavel the remaining 10,000. The nearest neighor classifier will take a test image, compare it to every single one of the training images, and predict the label of the closest training image. In the image above and ont he right you can see an example result of such a procedure for 10 examples test images. Notice that in only about 3 our of the 10 examples an image of the same class in retrieved. For example, in the 8th row of the nearest training image to the horse head is a red car, presumably due to the strong black background.

You may have noticed that we left unspecificed the details of exactly how we compare two images, which in this case are just two blocks of 32 x 32 x 3. One of the simpliest possibilities is to compare the images pixels by pixel and add up all the differences. In other words, given two images and representing them as a vector $I_1, I_2$, a reasonable choice for comparing them might be the **L1 distance**:


$$d_1 (I_1, I_2) = \sum_{p} \left| I^p_1 - I^p_2 \right|$$

Where the sum is taken over all pixels. Here is the procedure visualized:

___

![](http://cs231n.github.io/assets/nneg.jpeg)

* An example of using pixel-wise difference to compare two images with L1 distaince. Two images are substracted elementwise and then all differences are added up to a signle number. If ywo images are identical the result will be zero. But if images are different the result will be large 

_____

Let's also look at how we might implement the classifier in code. First, let's load the CIFAR-10 data into memory as 4 arrays: the training/labels and the test/labels. In the code, ```Xtr``` (of size 50,000 x 32 x 32 x 3) holds all the images in the training set, and a corresponding 1-dimensional array ```Ytr``` (of length 50,000) holds the training labels (from 0 to 9):

In [7]:
(Xtr, Ytr), (Xte, Yte) = cifar10.load_data()

100.0% 170500096 / 170498071


('Succesfully downloaded', 'cifar-10-python.tar.gz', 170498071, 'bytes.')
File Extracted in Current Directory


In [22]:
print(Xtr.shape) # Returns a shape 50,000 images, 32 x 32 x 3
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Reshape to 3072 
print(Xtr_rows.shape) # Xtr_rows become 50,000 x 3072

print(Xte.shape)
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows become 10,000 x 3072
print(Xte_rows.shape)
print(Ytr.shape)

(50000, 32, 32, 3)
(50000, 3072)
(10000, 32, 32, 3)
(10000, 3072)
(50000,)


In [32]:
# Define our NearestNeigbor Class

class NearestNeigbor(object):
    def __init__(self):
        pass
    
    def train(self, X, y ):
        """X is N x D where each row is an example. Y is 1-dimension of size N"""
        self.Xtr = X
        self.ytr = y
    
    def predict(self, X):
        """X is N x D where each row is an example we wish to predict label for"""
        # Number of test sets 50,000
        num_test = X.shape[0]
        # lets make sure that the output type matches the input type
        Ypred = np.zeros(num_test,dtype=self.ytr.dtype)
        
        # loop over all test rows
        for i in range(num_test):
            # find the nearest training image to the i'th test image
            # using the L1 distance (sum of absolute value differences)
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            min_index = np.argmin(distances)
            Ypred[i] = self.ytr[min_index]
        return Ypred
            

In [34]:
nn = NearestNeigbor()
nn.train(Xtr_rows,Ytr)
Yte_predict = nn.predict(Xte)

print('accuracy : %f' % (np.mean(Yte_predict == Yte)))

if you ran this code, you would see that this classifier only achieves a __38.6%__ on CIFAR-10. That's more impressive than guessing at random, but is nowhere near human performance.

__The choice of distance__. There are many other ways of computing distances between vectors. Another common choice could be to instead use the __L2 distance__, which has the geometric interpretation of computing the euclidean distance between two vectors. The distance takes the form:

$$d_2 (I_1, I_2) = \sqrt{\sum_{p} \left( I^p_1 - I^p_2 \right)^2}$$

In other words we would be computing the pixelwise difference as before, but this time we square all of them, add them up and finally take the square root. In numpy, using the code from above we would need to only replace a single line of code

```python 

distance = np.sqrt(np.sum(np.square(sel.Xtr - X[i,:]), axis = 1))

```

Note that I included the ```n.sqrt``` call above, but in practical nearest neighbor application we could leave out the square root operation, because the square root is a _monotonic function_. That is, it scales the absolute sizes of the distances but preserves the ordering, so the nearest neighbors with or without it are identical. 

L1 vs L2. It is interesting to consider differences between the two metrics. In particular the L2 distance is much more unforgiving than the L1 distance when it comes to the differences between two vectors. That is, the l2, distance perfers many medium disagreements to one big one. L1 and L2 distances are the most commonly used special cases of a p-norm

## K- Nearest Neigbour Classifier

You may have noticed that it is strange to only use the label of the enarest image when we wish to make a prediction. Indeed, it is almost always the case that one can do better by using what's called a *K-Nearest Neighbor Classifier*. The idea is very simple, instead of finding the closest single image in the training set, we find the top k closest images, and have them vote on the label of the test image. In particular, when  k = 1, we recover the Nearest Neigbor classifier. Intuitvely, higher values of K have a smoothing effect that makes the classifier more resistant to outliers

![](http://cs231n.github.io/assets/knn.jpeg)

 * An example of the difference between Nearest Neighbor and a 5-Nearest Neighbor classifier, using 2-dimensional points and 3 classes (red, blue, green). The colored regions show the decision boundaries induced by the classifier with an L2 distance. The white regions show points that are ambiguously classified (i.e. class votes are tied for at least two classes). Notice that in the case of a NN classifier, outlier datapoints (e.g. green point in the middle of a cloud of blue points) create small islands of likely incorrect predictions, while the 5-NN classifier smooths over these irregularities, likely leading to better generalization on the test data (not shown). Also note that the gray regions in the 5-NN image are caused by ties in the votes among the nearest neighbors (e.g. 2 neighbors are red, next two neighbors are blue, last neighbor is green).
 

## Validation set for Hyperparameter tuning